# 各縣市補習班資訊

資料來源： [直轄市及各縣市短期補習班資訊管理系統](http://bsb.edu.tw/)

In [129]:
import urllib.request
from urllib.parse import quote_plus
from bs4 import BeautifulSoup as bs
import json

In [21]:
host = 'http://bsb.edu.tw/afterschool/opendata/{}'
url = 'http://bsb.edu.tw/afterschool/opendata/afterschool_list.jsp'
f = urllib.request.urlopen(url);
soup = bs(f.read(), 'lxml')

In [31]:
data = {}
for a in soup.select('div > a'):
    json_url = host.format(a['href'])
    raw = urllib.request.urlopen(json_url).read().decode('utf-8');
    data[a.text.strip()] = json.loads(raw)

In [32]:
data.keys()

dict_keys(['屏東縣', '新竹市', '宜蘭縣', '苗栗縣', '新北市', '嘉義市', '高雄市', '彰化縣', '澎湖縣', '南投縣', '台中市', '嘉義縣', '桃園市', '台北市', '新竹縣', '臺東縣', '花蓮縣', '基隆市', '雲林縣', '台南市', '連江縣', '金門縣'])

In [81]:
data['新竹縣'][0]

{'各地短期補習班數量': '355',
 '地區縣市': '新竹縣政府',
 '地址': '竹北市東海里5鄰東海二街200巷1弄13號',
 '短期補習班名稱': '私立立勝短期補習班',
 '短期補習班類別': '外語類',
 '立案時間': '2005-12-01'}

In [36]:
items = []

for k, v in data.items():
    items += v

In [41]:
cats = [item['短期補習班類別'] for item in items]
cats = list(set(cats))
print(cats)

['資訊類', '美術、書法、攝影、美工設計、圍棋類', '縫紉類', '音樂、舞蹈類', '文理類', '法政類', '瑜珈類', '速讀類', '美容、美髮理髮類', '其他類', '電機、汽車修護、建築、工藝、製圖類', '商類：珠算、心算、會計', '外語類', '家政、插花、烹飪類']


In [48]:
from collections import Counter

city_cats = {}
for city, schools in data.items():
    city_cats[city] = dict(Counter([s['短期補習班類別'] for s in schools]))

In [53]:
with open('city_cats.json', 'w') as fp:
    json.dump(city_cats, fp, ensure_ascii=False, indent=2)

In [55]:
dict(Counter([d['短期補習班類別'] for d in data['新竹市']]))

{'其他類': 1,
 '商類：珠算、心算、會計': 1,
 '外語類': 123,
 '文理類': 246,
 '法政類': 3,
 '美容、美髮理髮類': 4,
 '美術、書法、攝影、美工設計、圍棋類': 6,
 '資訊類': 7,
 '電機、汽車修護、建築、工藝、製圖類': 1,
 '音樂、舞蹈類': 18}

# 105 學年度在學人口統計

資料來源： [教育部統計處](https://stats.moe.gov.tw)

In [60]:
high_school_str = '''高級中等學校學生數-依設立別查詢
★學年別	★縣市別	總計
105學年	新北市	94,332
105學年	臺北市	108,182
105學年	桃園市	81,645
105學年	臺中市	105,226
105學年	臺南市	61,815
105學年	高雄市	89,805
105學年	宜蘭縣	13,901
105學年	新竹縣	14,678
105學年	苗栗縣	18,386
105學年	彰化縣	42,546
105學年	南投縣	15,643
105學年	雲林縣	21,463
105學年	嘉義縣	10,434
105學年	屏東縣	22,018
105學年	臺東縣	6,388
105學年	花蓮縣	11,284
105學年	澎湖縣	2,733
105學年	基隆市	12,768
105學年	新竹市	20,731
105學年	嘉義市	19,698
105學年	金門縣	2,112
105學年	連江縣	324'''

middle_str = '''國民中小學學生數-依學校所在地區查詢
★學年別	★縣市別	國小	國中
105學年	新北市	195,706	106,199
105學年	臺北市	116,496	69,178
105學年	桃園市	124,604	70,463
105學年	臺中市	153,062	89,408
105學年	臺南市	89,475	53,089
105學年	高雄市	129,926	76,167
105學年	宜蘭縣	22,349	14,783
105學年	新竹縣	35,692	18,044
105學年	苗栗縣	28,579	16,836
105學年	彰化縣	65,423	39,679
105學年	南投縣	24,180	15,716
105學年	雲林縣	33,997	22,308
105學年	嘉義縣	20,810	13,153
105學年	屏東縣	37,482	24,014
105學年	臺東縣	10,702	6,682
105學年	花蓮縣	15,914	10,165
105學年	澎湖縣	3,756	2,452
105學年	基隆市	16,654	10,279
105學年	新竹市	29,534	15,683
105學年	嘉義市	15,468	10,634
105學年	金門縣	3,618	2,023
105學年	連江縣	455	249'''

In [69]:
city_population = {}
for entry in high_school_str.split('\n')[2:]:
    city = entry.split('\t')[1]
    number = int(entry.split('\t')[2].replace(',', ''))
    city_population[city] = {'senior': number}

for entry in middle_str.split('\n')[2:]:
    city = entry.split('\t')[1] 
    primary = int(entry.split('\t')[2].replace(',', ''))
    junior = int(entry.split('\t')[3].replace(',', ''))
    city_population[city]['primary'] = primary
    city_population[city]['junior'] = junior
    city_population[city]['total'] = primary + junior + city_population[city]['senior']

In [73]:
with open('city_population_105.json', 'w') as fp:
    json.dump(city_population, fp, ensure_ascii=False, indent=2)

# 密度計算

文理補習班數量 / 在學人口

In [78]:
city_cram_density = {}
for city, cats, in city_cats.items():
    city_name = city.replace('台', '臺')
    density = 0
    cats_num = 0
    if '文理類' in cats:
        cats_num = cats['文理類'] 
        density = cats_num / city_population[city_name]['total'] * 1000
    city_cram_density[city] = {'crams': cats_num, 'students': city_population[city_name]['total'], 'density': density }

In [80]:
with open('city_cram_density.json', 'w') as fp:
    json.dump(city_cram_density, fp, ensure_ascii=False, indent=2)

# 新竹學校附近補習班分布狀況
距離學校 1.5 公里內的補習班

學校資料： [新竹市教育網](http://www.hc.edu.tw/school/BasicData/BasicData.aspx)

In [199]:
import pandas as pd

df_city = pd.read_csv('raw_data/竹市補習班.csv')
df_county = pd.read_csv('raw_data/竹縣補習班.csv')

In [200]:
# 新竹市補習班資料
df_city.head()

,緯度,經度,補習班名稱,電話,核准科目名稱,補習班住址,立案文號,立案日期,備註
0,24.805791,120.962204,新竹市私立蕎立小學堂文理短期補習班,NaN,美語數學自然,新竹市北大路319、319-1號2-3樓,府教社字第1060134979號,9/14/2017,NaN
1,24.784052,121.020338,新竹市私立迦南文理短期補習班,NaN,國語其他其他,新竹市光復路一段531巷80-1號1樓,府教社字第1060094304號,7/10/2017,NaN
2,24.795743,120.948937,新竹市私立青庭美語短期補習班,0922979736,英語,新竹市中華路四段226巷12號2樓及3樓,府教社字第1060083425號,6/6/2017,NaN
3,24.817693,120.973670,新竹市私立華新文理短期補習班,(03)5335001,英文數學理化,新竹市光華北街29號1樓至3樓,府教社字第1060058289號,4/21/2017,NaN
4,24.794496,120.977534,新竹市私立華盛頓短期補習班學府分校,(03)5753939,英語英語,新竹市學府路31號,府教社字第1060051959號,3/29/2017,NaN


In [201]:
# 新竹縣補習班資料
df_county.head()

,緯度,經度,補習班名稱,電話
0,24.812391,121.030753,愛思可樂教育事業股份有限公司附設新竹縣私立樂研文理短期補習班,(03)5500105
1,24.815008,121.020831,丞瑞實業有限公司附設新竹縣私立丞瑞身心適能技藝短期補習班,(03)6214168
2,24.827898,121.029222,新竹縣私立芯學淵文理短期補習班,(03)6578307
3,24.822343,121.015305,新竹縣私立英王文理短期補習班,(03)5528490
4,24.824950,121.032623,新竹縣私立奇玉美語文理短期補習班,0932134670


In [202]:
with open('df_city.json', 'w', encoding='utf-8') as file:
    df_city.to_json(file,orient='records',force_ascii=False)
with open('df_county.json', 'w', encoding='utf-8') as file:
    df_county.to_json(file,orient='records',force_ascii=False)

In [205]:
cramschool_items = json.load(open('df_county.json')) + json.load(open('df_city.json'))#

In [167]:
print(cramschool_items[0])
print(cramschool_items[-1])

{'緯度': 24.8057911, '補習班名稱': '新竹市私立蕎立小學堂文理短期補習班', '立案文號': '府教社字第1060134979號', '補習班住址': '新竹市北大路319、319-1號2-3樓', '立案日期': '9/14/2017', '備註': None, '經度': 120.9622035, '核准科目名稱': '美語數學自然', '電話': None}
{'緯度': '24.7365549', '經度': ' 121.08755229999997', '補習班名稱': '私立自強文理短期補習班', '電話': '(03)5963863 ,5955798\u3000'}


In [108]:
# 新竹市學校地址資料

schools_str = '''
1	成德高中	黃小芳	5258748	5266049	30047崧嶺路128巷38號
2	香山高中	謝大才	5384332	5308149	30093香山里元培街124號
3	建功高中	傅瑞琪	5745892	5737584	30070建功二路17號
4	建華國中	黃信騰	5238075	5266020	30068建華里2鄰學府路2號
5	培英國中	丁淑觀	5721301	5726578	30068東山里學府路4號
6	光華國中	宋雨親	5316605	5340698	30053光華北街10號
7	育賢國中	黃淑文	5223075	5266047	30063南大路569號
8	光武國中	林茂成	5773934	5786856	30072光復路一段512號
9	南華國中	陳彥宇	5362204	5362671	30058海濱路55號
10	富禮國中	宋隆文	5374793	5371451	30094富禮街16號
11	三民國中	洪碧霜	5339825	5337553	30041自由路95巷15號
12	內湖國中	張慧媛	5373484	5371802	30095五福路一段12號
13	虎林國中	鄭明谷	5309433	5397421	30090延平路二段76號
14	新科國中	彭元豐	6686387	6686386	30074光復路一段89巷88號
15	竹光國中	潘致惠	5246683	5244967	30044和平路1號
01	國立清華大學	5715131	5710582	光復路二段101號
02	 國立交通大學 	5712121	5712794	大學路1001號
04	 國立園區實中 	5777011	5781813	介壽路300號
05	 國立清華大學附小 	5282420	5253470	北區四維路47號
06	 國立新竹高中 	5736666	5736699	東區學府路36號
07	 國立新竹高商 	5714104	5723212	學府路128號
08	 國立新竹女中 	5456611	5425894	中華路二段270號
09	 國立新竹高工 	5322175	5330381	中華路二段2號
10	 私立中華大學 	5374281	5373771	香山區五福路二段707號
11	 私立玄奘大學 	5302255	5319208	香山區東香里玄奘路48號
12	 私立元培科技大學 	5381183	5385353	香山區新竹市元培街306號
13	 私立世界高中 	5783271	5783275	東區光復路一段257號
14	 私立磐石中學 	5223946	5243195	北區西大路683號
15	 私立光復中學 	5753699	5720771	光復路二段153號
16	 私立曙光女中 	5325709	5422125	東區北大路61號
17	 私立康橋中小學 	5192000	5192222	國家藝術園區藝術路二號
18	 私立曙光小學 	5328283	5425787	東區北大路70號
19	 新竹荷蘭國際學校 	5267837	5248261	牛埔東路290號
20	 新竹美國學校 	5203211	5296482	國家藝術園區藝術路2號
21	 亞太美國學校 	5717070	5721148	光復路二段151號3F
'''

In [139]:
def is_high_school(name):
    for keyword in ['高中', '國中', '中學', '實中', '附中', '女中', '高商', '高工']:
        if keyword in name:
            return True
    return False

school_address = {}

# 把小學、大學過濾掉
for line in schools_str.split('\n')[1:]:
    if len(line)==0: continue
    row = line.split('\t')
    if is_high_school(row[1]):
        school_address[row[1].strip()] =  { 'address': row[-1].strip() }

In [183]:
## use google api to get the coordinate of an address
def get_coord(address):
    google_map_url = 'http://maps.google.com/maps/api/geocode/json?address={}'
    f = urllib.request.urlopen(google_map_url.format(quote_plus(address)))
    result = json.loads(f.read().decode('utf-8'))
    return result['results'][0]['geometry']['location']

In [142]:
# get coord of each school in Hsinchu
for school, value in school_address.items():
    value['coord'] = get_coord(value['address'])

In [170]:
# manually added 竹北高中
school_address['國立竹北高中'] = {'address': '302新竹縣竹北市中央路3號',
  'coord': {'lat': 24.8367557, 'lng': 121.0019671}}

In [185]:
from math import sin, cos, sqrt, atan2, radians

# calculate distance between two coords
def get_distance(_lat1, _lon1, _lat2, _lon2):
    # approximate radius of earth in km
    R = 6373.0
    lat1 = radians(_lat1)
    lon1 = radians(_lon1)
    lat2 = radians(_lat2)
    lon2 = radians(_lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [212]:
# 計算學校附近的補習班

school_near_cram = {}
for school, value in school_address.items():
    school_near_cram[school] = []
    lat_s = float(value['coord']['lat'])
    lon_s = float(value['coord']['lng'])
    
    for cram in cramschool_items:
        lat_c = float(cram['緯度'])
        lon_c = float(cram['經度'])
        
        if get_distance(lat_s, lon_s, lat_c, lon_c) <= 1.5:
            school_near_cram[school].append(cram['補習班名稱'])

In [226]:
for s, v in school_near_cram.items():
    print('{} {}'.format(s,len(v)))

私立光復中學 49
光華國中 113
國立新竹女中 211
建華國中 133
新科國中 59
三民國中 156
竹光國中 0
私立磐石中學 91
成德高中 41
國立新竹高商 77
國立園區實中 61
光武國中 62
富禮國中 1
培英國中 127
香山高中 8
南華國中 6
建功高中 25
育賢國中 61
國立竹北高中 79
國立新竹高工 0
內湖國中 5
私立曙光女中 189
私立世界高中 63
國立新竹高中 117
虎林國中 21
